In [1]:
import sys
import os
from glob import glob
from itertools import combinations

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


# Avoids warnings
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"
import tensorflow as tf

import ScalableLib.classifier.Multiband as multiband
from ScalableLib.classifier.CustomLayers import SauceLayer, LastRelevantLayer, ApplyMask, MeanMagLayer, MeanColorLayer

2024-12-08 20:15:53.052240: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-08 20:15:53.052272: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-08 20:15:53.053246: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-08 20:15:54.073672: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


the filtering of the times must be done in the spine, not in each band. Spit all the times then slect the best one. 

In [2]:
# To see if the system recognises the GPU
device = 0
devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.set_visible_devices(devices[device], 'GPU')
tf.config.experimental.set_memory_growth(device=devices[device], enable=True)

device_name = tf.config.experimental.get_device_details(devices[device])['device_name']
print("Using {}".format(device_name))

Using NVIDIA GeForce RTX 3080


Find the different folds and train a model using the stored data.

In [3]:
survey = 'ZTF'
path = os.path.join('../../../CV_c/02_CreateRecords/', survey, 'Folds/Fold_*',)
folds = glob(path)
folds.sort()
folds

['../../../CV_c/02_CreateRecords/ZTF/Folds/Fold_1',
 '../../../CV_c/02_CreateRecords/ZTF/Folds/Fold_2',
 '../../../CV_c/02_CreateRecords/ZTF/Folds/Fold_3',
 '../../../CV_c/02_CreateRecords/ZTF/Folds/Fold_4',
 '../../../CV_c/02_CreateRecords/ZTF/Folds/Fold_5',
 '../../../CV_c/02_CreateRecords/ZTF/Folds/Fold_6',
 '../../../CV_c/02_CreateRecords/ZTF/Folds/Fold_7']

Create folder results

In [4]:
if not os.path.exists('./Results'):
    os.mkdir('./Results')


Define the arguments for all the models.

In [5]:
train_args = {
            'hidden_size_bands':[128,128, 128],
            'hidden_size_central':[128, 128],
            'fc_layers_bands':[128,128,128],
            'fc_layers_central':[128,128,128], # Neurons of each layer
            'regression_size':[128, 128],#each element is a layer with that size.
            'buffer_size':10000,
            'epochs':1000,
            'num_threads':7,
            'batch_size':128,
            'dropout':0.30,
            'lr':[[5e-3]*2, 2.5e-3], # [[band1, band2], central]
            'val_steps':50,
            'max_to_keep':0, # Not Used 
            'steps_wait':500, 
            'use_class_weights':True,# Not Used as intended, for initialization
            'mode' : 'classifier'
            }
loss_weights = {'Class':1.0}

callbacks_args = {'patience': 20,
                  'mode':'max',
                  'restore_best_weights':True,
                  'min_delta': 0.001
                 }
train_args_specific={
                    'phys_params': [],
                    'use_output_bands' : True,  # Working
                    'use_output_central' : False, # Not used
                    'use_common_layers' : False, # NOT Working
                    'bidirectional_central' : False,# Working
                    'bidirectional_band' : False,# Not Working
                    'layer_norm_params' : None, # Used to normalyze common layers
                    'use_gated_common' : False, # Working
                    'l1':0.0,
                    'l2':0.0,  
                    'N_skip': 3, # Cannot be greater than the number of timesteps
                    'use_raw_input_central': False,
                    'train_steps_central' : 2,
                    'print_report' : True,
                    'loss_weights_central' : loss_weights,
                    'callbacks_args':callbacks_args
                    }



Create the scapulars and coonect them at the end

In [6]:
def compute_FinalMeanMags(output, length):

    batch_size = tf.shape(output)[0]
    indices = tf.stack([tf.range(0, batch_size), length], axis=1)
    final_MeanMags = tf.gather_nd(output, indices)
    return final_MeanMags

def compute_ColorMatrix(n_bands):
    """ Obtains the matrix to compute the colors (magnitude differences)
    between all the bands. In the 2-band scenario it reduces to [[1, -1]].
    """
    NN = n_bands
    # Compute all possible combinations
    combs = set(combinations(np.arange(NN), 2))
    combs = [list(i) for i in combs]
    combs = tf.constant(combs)

    # Compute the index of the first elements
    v1 = tf.reshape(combs[:, 0], [-1, 1])
    v1b = tf.reshape(tf.range(tf.shape(combs)[0]), [-1, 1])
    v1 = tf.concat([v1b, v1], axis=1)
    # Compute the index of the second elements
    v2 = tf.reshape(combs[:, 1], [-1, 1])
    v1b = tf.reshape(tf.range(tf.shape(combs)[0]), [-1, 1])
    v2 = tf.concat([v1b, v2], axis=1)

    # Concatenate and cast to int64
    indices = tf.concat([v1, v2], axis=0)
    indices = tf.cast(indices, tf.int64)

    # First part are the first elements
    u1 = tf.ones(tf.shape(v1)[0])
    # Second part are the subtracted elements
    u2 = -tf.ones(tf.shape(v1)[0])
    updates = tf.concat([u1, u2], axis=0)

    # Compute the shape #combinations x bands
    shape = tf.stack([tf.shape(v1)[0], NN], axis=0)
    shape = tf.cast(shape, tf.int64)

    # Obtain the matrix
    diff_matrix = tf.scatter_nd(indices, updates, shape)

    # Transpose it to perform MatMult afterwards
    diff_matrix = tf.transpose(diff_matrix)
    return diff_matrix

In [7]:
class LastRelevantLayer_mod(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(LastRelevantLayer_mod, self).__init__(**kwargs)
        self.supports_masking = True

    def call(self, output, length):
        '''Get the last relevant output from the network'''
        batch_size = tf.shape(output)[0]
        max_length = tf.shape(output)[1]
        out_size = tf.shape(output)[2]

        index = tf.range(0, batch_size) * max_length + (length - 1)
        flat = tf.reshape(output, [-1, out_size])
        relevant = tf.gather(flat, index)
        
        relevant = tf.reshape(relevant, [batch_size, output.shape[2]])
        return relevant
 

def creat_split_models():
    tf.keras.backend.clear_session()
    inputs = {}
    keys =new.dataset_test.element_spec[0].keys()
    for key in keys:
        inputs[key] = tf.keras.layers.Input(shape=new.dataset_test.element_spec[0][key].shape[1:],
                                            dtype=new.dataset_test.element_spec[0][key].dtype,
                                            name=key
                                            )
    out = {}
    sauces = []
    Mean_Mags = []
    
    for model in range(len(new.models)):
        # Get the single band outputs
        new.models[model].trainable = True
        out[model] = new.models[model].layers[2].output
        slice_model = tf.keras.Model(inputs=new.models[model].inputs, outputs=out[model])

        out_model = slice_model(inputs)
        # Get the last relevant per band per output
        lasts = []
        for i in range(len(out_model)):
            last_relevant = LastRelevantLayer_mod()(out_model[i], inputs['N_'+str(model)])
            lasts.append(last_relevant)
        # Create a Sauce layer per band
        size = len(lasts)
        sauce = SauceLayer(size, name='Sauce_'+str(model))(lasts)
        sauces.append(sauce)

        # Compute the mean mags
        MeanMag = MeanMagLayer(new.w,
                               name='MeanMag_'+str(model)
                               )
        mean_mags = MeanMag(inputs['input_LC_'+str(model)],
                            inputs['N_'+str(model)],
                            inputs['M0_'+str(model)],
                            )        
        Mean_Mags.append(mean_mags)

    final_MeanMags = [compute_FinalMeanMags(Mean_Mags[i], inputs['N_'+str(i)]) for i in range(new.n_bands)]
    final_MeanMags = tf.stack(final_MeanMags, axis=1)
    color_matrix = compute_ColorMatrix(new.n_bands)
    colors = tf.matmul(final_MeanMags, color_matrix)
        
    # Stack the outputs
    embedding = tf.keras.layers.Concatenate(axis=1, name='Concat_Sauces')(sauces)

    # Stack the colors
    embedding =  tf.keras.layers.Concatenate(axis=-1, name='Concat_Colors')([embedding, colors])
    # Add dense layers with dropout
    sizes = train_args['fc_layers_central']
    proyections = embedding
    for b in range(len(sizes)):
        proyections = tf.keras.layers.Dense(sizes[b],
                                        activation=None,
                                        use_bias=False,
                                        )(proyections)
        proyections = tf.keras.layers.BatchNormalization()(proyections, training=True)
        proyections = tf.keras.activations.relu(proyections)

        proyections = tf.keras.layers.Dropout(new.dropout)(proyections)

    predictions_prob = tf.keras.layers.Dense(new.num_classes,
                                             activation='softmax',
                                             use_bias=True,
                                             name='Predictions',
    #                                          bias_initializer=tf.constant_initializer(1.0/self.numpy_weights),
                                             )(proyections)
    outputs = {
            'Class': predictions_prob,

            }
    model = tf.keras.Model(inputs = inputs, outputs=outputs)
    optimizer = new.optimizers[0]
    loss = 'categorical_crossentropy'
    model.compile(optimizer=optimizer, loss=loss, metrics='accuracy')

    return model

In [8]:
sauces = []
for fold in folds:
    tf.keras.backend.clear_session()
    # Set the fold path
    base_dir = fold+'/'
    
    # Set the save path for this fold. Create folder if needed
    path_results_fold = fold.replace(f'../../../CV_c/02_CreateRecords/{survey}', '.').replace('/Folds/', '/Results/')

    if not os.path.exists(path_results_fold):
        os.mkdir(path_results_fold)    

    train_args_specific['save_dir'] = path_results_fold
    train_args_specific['metadata_pre_path'] = os.path.join(fold, 'metadata_preprocess.json')
    train_args_specific['path_scalers'] =  os.path.join(fold,'scalers.pkl')
    # Define the train args
    train_args = {**train_args, **train_args_specific}

    train_files = os.path.join(fold, 'train/*.tfrecord')
    val_files = os.path.join(fold, 'val/*.tfrecord')
    test_files = os.path.join(fold, 'test/*.tfrecord')
    
    new = multiband.Network()    
    new.train(train_args, train_files, val_files, test_files)
    model = creat_split_models()
    
    es = tf.keras.callbacks.EarlyStopping(**callbacks_args)
    model.fit(new.dataset_train, validation_data=new.dataset_val, epochs=200,callbacks = [es] )
    
    # Store the alpha coefficients per fold
    sauces_ = {i.name:i for i in model.layers if 'Sauce_' in i.name}
    scales = {key:tf.nn.softmax(sauces_[key].scale).numpy() for key in sauces_.keys()}
    sauces.append(scales)
    # Test
    dfs = []
    for batch in new.dataset_test:
        prediction = model(batch[0])
        y_pred = prediction['Class'].numpy().argmax(axis=1)
        y_pred = [new.trans[i] for i in y_pred]
        ID = batch[0]['ID'].numpy()

        y_true = batch[1]['Class'].numpy().argmax(axis=1)
        y_true = [new.trans[i] for i in y_true]

        df = pd.DataFrame(np.array([ID, y_pred, y_true]).transpose(), columns=['ID', 'Class', 'Pred'])
        df.ID = df.ID.str.decode('UTF-8')
        dfs.append(df)
    dfs = pd.concat(dfs, axis=0)
    dfs.to_csv(path_results_fold+'/Classification_test.dat', index=False, index_label=False)
    
pd.DataFrame(sauces).to_csv(path_results_fold+'/sauces.dat', index=False, index_label=False)    


./Results/Fold_1/Models/20241208-2015
Epoch 1/200


I0000 00:00:1733706996.863128  718217 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


241/241 [==============================] - 66s 174ms/step - loss: 0.9944 - accuracy: 0.6657 - val_loss: 0.5717 - val_accuracy: 0.8076
Epoch 2/200
241/241 [==============================] - 37s 154ms/step - loss: 0.6132 - accuracy: 0.7926 - val_loss: 0.4978 - val_accuracy: 0.8312
Epoch 3/200
241/241 [==============================] - 31s 127ms/step - loss: 0.5298 - accuracy: 0.8204 - val_loss: 0.5410 - val_accuracy: 0.8140
Epoch 4/200
241/241 [==============================] - 32s 131ms/step - loss: 0.4854 - accuracy: 0.8348 - val_loss: 0.4201 - val_accuracy: 0.8565
Epoch 5/200
241/241 [==============================] - 33s 136ms/step - loss: 0.4363 - accuracy: 0.8526 - val_loss: 0.4070 - val_accuracy: 0.8620
Epoch 6/200
241/241 [==============================] - 30s 126ms/step - loss: 0.4110 - accuracy: 0.8601 - val_loss: 0.3901 - val_accuracy: 0.8653
Epoch 7/200
241/241 [==============================] - 30s 125ms/step - loss: 0.3880 - accuracy: 0.8699 - val_loss: 0.3800 - val_accurac


KeyboardInterrupt



In [ ]:
1